In [14]:
import datetime as dt
from datetime import datetime, timedelta
from vnstock import Vnstock
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import style


In [15]:
ticker = 'BID'

stock = Vnstock().stock (symbol = ticker , source = 'TCBS')

stock_DF = stock.quote.history(start = '2024-08-20', end = '2025-08-20', interval = '1D')

print(stock_DF)


2025-08-22 17:51:19 - vnstock.common.data.data_explorer - INFO - TCBS không cung cấp thông tin danh sách. Dữ liệu tự động trả về từ VCI.


          time   open   high    low  close    volume
0   2024-08-20  39.92  40.66  39.79  40.62   4363152
1   2024-08-21  40.86  41.98  40.53  41.82   6176457
2   2024-08-22  42.15  42.39  41.24  41.57   2741504
3   2024-08-23  41.48  41.98  41.40  41.73   2275733
4   2024-08-26  41.98  42.06  41.40  41.48   2249313
..         ...    ...    ...    ...    ...       ...
245 2025-08-14  41.60  42.65  41.15  42.15  15184586
246 2025-08-15  42.60  42.60  40.70  41.00  14384373
247 2025-08-18  41.05  41.30  40.60  40.80   7520468
248 2025-08-19  40.90  41.50  40.70  40.85   9932709
249 2025-08-20  41.00  41.85  39.60  40.70  14551914

[250 rows x 6 columns]


In [16]:
stock_DF['time'] = pd.to_datetime(stock_DF['time'])

#Shift(1): Yesterday value, Shift(2): Value 2 days ago, Shift(-1): Tomorrow value 
#np.log = numpy library, log normal. 

stock_DF ['log normal return'] = np.log(stock_DF['close'] / stock_DF['close'].shift(1))*100

print(stock_DF)


          time   open   high    low  close    volume  log normal return
0   2024-08-20  39.92  40.66  39.79  40.62   4363152                NaN
1   2024-08-21  40.86  41.98  40.53  41.82   6176457           2.911414
2   2024-08-22  42.15  42.39  41.24  41.57   2741504          -0.599594
3   2024-08-23  41.48  41.98  41.40  41.73   2275733           0.384154
4   2024-08-26  41.98  42.06  41.40  41.48   2249313          -0.600891
..         ...    ...    ...    ...    ...       ...                ...
245 2025-08-14  41.60  42.65  41.15  42.15  15184586           1.916226
246 2025-08-15  42.60  42.60  40.70  41.00  14384373          -2.766262
247 2025-08-18  41.05  41.30  40.60  40.80   7520468          -0.488999
248 2025-08-19  40.90  41.50  40.70  40.85   9932709           0.122474
249 2025-08-20  41.00  41.85  39.60  40.70  14551914          -0.367873

[250 rows x 7 columns]


In [17]:
print(stock_DF.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 250 entries, 0 to 249
Data columns (total 7 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   time               250 non-null    datetime64[ns]
 1   open               250 non-null    float64       
 2   high               250 non-null    float64       
 3   low                250 non-null    float64       
 4   close              250 non-null    float64       
 5   volume             250 non-null    int64         
 6   log normal return  249 non-null    float64       
dtypes: datetime64[ns](1), float64(5), int64(1)
memory usage: 13.8 KB
None


In [18]:
#Eliminate, Keep, Add more columns into DataFrame

stock_DF = stock_DF.drop(columns = ['open', 'high', 'low', 'volume'])

print(stock_DF)

          time  close  log normal return
0   2024-08-20  40.62                NaN
1   2024-08-21  41.82           2.911414
2   2024-08-22  41.57          -0.599594
3   2024-08-23  41.73           0.384154
4   2024-08-26  41.48          -0.600891
..         ...    ...                ...
245 2025-08-14  42.15           1.916226
246 2025-08-15  41.00          -2.766262
247 2025-08-18  40.80          -0.488999
248 2025-08-19  40.85           0.122474
249 2025-08-20  40.70          -0.367873

[250 rows x 3 columns]


In [19]:
import pandas as pd
import numpy as np

ticker = ['HDB', 'ACB', 'VNM', 'BID', 'HPG', 'FPT']

#Big folder to put all the CSV in. Then at the end, pd.concat to stack them together in 1 DataFrame
all_quotes = []

#Divide each in separate. If want to call separately, use code: per_ticker['FPT'].head() or per_ticker['HDB'].tail()
per_ticker = {}

for t in ticker: 

    s = Vnstock().stock (symbol = t, source = 'VCI')

    df = s.quote.history (start = '2024-08-18', end = '2025-08-18', interval = '1D')

    df['time'] = pd.to_datetime(df['time'])

    df.sort_values(by ='time',ascending=False,inplace=True)

    df['Ticker'] = t

    cols = ['Ticker'] + [c for c in df if c != 'Ticker']

    df = df[cols]

    df['log normal return'] = np.log(df['close'] / df['close'].shift(-1))*100

    df = df.drop(columns = ['open', 'high', 'low', 'volume'])

    per_ticker [t] = df

    all_quotes.append(df)


cols = ['Ticker'] + [c for c in df if c != 'Ticker']
df = df[cols]

all_DF = pd.concat(all_quotes, ignore_index = True)
all_DF


,Ticker,time,close,log normal return
0,HDB,2025-08-18,30.50,-1.626052
1,HDB,2025-08-15,31.00,0.161421
2,HDB,2025-08-14,30.95,6.680280
3,HDB,2025-08-13,28.95,0.172861
4,HDB,2025-08-12,28.90,3.521491
...,...,...,...,...
1489,FPT,2024-08-23,113.97,-0.525073
1490,FPT,2024-08-22,114.57,0.227193
1491,FPT,2024-08-21,114.31,1.507234
1492,FPT,2024-08-20,112.60,0.766697


In [20]:
# Put every ticker into 1 DataFrame where only appears their tickers, Annualized return, and Annualized STD
def summarize(group):
    log_return = group['log normal return'].dropna()
    daily_return = log_return.mean()
    daily_STD = log_return.std(ddof = 0) #ddof = 0 => use population STD (similar to Excel's var.p), use ddof = 1 if var.S

    return pd.Series({
        'Annualized_Return': daily_return *252, 

        'Annualized STD': daily_STD * np.sqrt(252)  
    })

summary = all_DF.groupby('Ticker'). apply(summarize). reset_index()

print(summary)


  Ticker  Annualized_Return  Annualized STD
0    ACB          30.226265       21.117571
1    BID           3.084779       22.477364
2    FPT         -10.772697       27.208900
3    HDB          30.911801       29.288419
4    HPG          28.093679       26.099112
5    VNM         -15.035197       20.671886


DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.


In [22]:
all_DF.groupby('Ticker')['close'].mean()

Ticker
ACB     21.363373
BID     38.625141
FPT    112.938876
HDB     22.840482
HPG     22.539197
VNM     60.639518
Name: close, dtype: float64

In [9]:
 #turn long format into wide format: one column per ticker 
returns_wide = all_DF.pivot(
    index = 'time', # cột đầu làm index benchmark
    columns = 'Ticker', 
    values = 'log normal return'
)
returns_wide.sort_index(ascending=False,inplace=True)
returns_wide = returns_wide /100.0

daily_cov = returns_wide.cov()*100

annualized_cov = daily_cov *252

print("Covariance Matrix (annualized): \n", daily_cov.dropna().round(6), "\n")

w = np.array([0.16, 0.16, 0.16, 0.16, 0.16, 0.16])
portfolio_var = w.T @ annualized_cov.values @ w
portfolio_STD = np.sqrt(portfolio_var)

print("Portfolio STD:", portfolio_STD, "\n")



Covariance Matrix (annualized): 
 Ticker       ACB       BID       FPT       HDB       HPG       VNM
Ticker                                                            
ACB     0.017768  0.012751  0.011324  0.014432  0.013077  0.008702
BID     0.012751  0.020130  0.013800  0.015771  0.014685  0.009657
FPT     0.011324  0.013800  0.029497  0.013811  0.014713  0.009176
HDB     0.014432  0.015771  0.013811  0.034178  0.015954  0.011846
HPG     0.013077  0.014685  0.014713  0.015954  0.027140  0.009963
VNM     0.008702  0.009657  0.009176  0.011846  0.009963  0.017026 

Portfolio STD: 1.8404565513106317 



In [10]:
corr_matrix = returns_wide.corr()
print(corr_matrix)

Ticker       ACB       BID       FPT       HDB       HPG       VNM
Ticker                                                            
ACB     1.000000  0.674198  0.494637  0.585652  0.595492  0.500302
BID     0.674198  1.000000  0.566337  0.601247  0.628280  0.521608
FPT     0.494637  0.566337  1.000000  0.434981  0.520017  0.409467
HDB     0.585652  0.601247  0.434981  1.000000  0.523827  0.491087
HPG     0.595492  0.628280  0.520017  0.523827  1.000000  0.463470
VNM     0.500302  0.521608  0.409467  0.491087  0.463470  1.000000


In [11]:
from scipy.optimize import minimize

returns_wide.dropna(how = 'any')


Ticker,ACB,BID,FPT,HDB,HPG,VNM
time,,,,,,
2025-08-18,0.018657,-0.004890,-0.008915,-0.016261,0.014185,-0.004930
2025-08-15,0.013270,-0.027663,-0.023393,0.001614,-0.007117,-0.011410
2025-08-14,0.067086,0.019162,-0.012446,0.066803,0.003552,0.009772
2025-08-13,0.010257,-0.003621,-0.022579,0.001729,-0.014135,-0.011391
2025-08-12,-0.006167,0.031826,0.010285,0.035215,0.012357,0.004866
...,...,...,...,...,...,...
2024-08-26,0.000000,-0.005768,-0.012094,0.000000,-0.001844,-0.020244
2024-08-23,0.001960,0.003842,-0.005251,0.000000,0.011582,0.009416
2024-08-22,0.000000,-0.005996,0.002272,0.000000,-0.015260,-0.014736
